<a href="https://colab.research.google.com/github/MarMarhoun/freelance_work/blob/main/side_projects/NLP_projs/eda_streamlit/ml_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project description:

Here is an example of how you can train a multiclassification task using PyTorch and then deploy the model using Gradio. First, let's start with training the model. We will use the FashionMNIST dataset, which is a dataset of 60,000 28x28 grayscale images of 10 fashion categories, along with a test set of 10,000 images. We will use a simple convolutional neural network (CNN) architecture for this task. Here's the code to train the model:

## Fisrt example

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Load the FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / (i + 1)}')

print('Finished Training')

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

Now, let's deploy the model using Gradio. Here's the code to create a simple web app using Gradio:

In [ ]:
import gradio as gr

# Define a function to make predictions with the model
def predict(image):
    # Preprocess the image
    image = image.resize((28, 28))
    image = image.convert('L')
    image = np.array(image)
    image = image.reshape(1, 1, 28, 28)
    image = image / 255.0
    # Make a prediction
    output = net(torch.from_numpy(image))
    _, predicted = torch.max(output.data, 1)
    # Return the predicted class
    return predicted.item()

# Create a Gradio interface
iface = gr.Interface(fn=predict,
                     inputs=gr.inputs.Image(type='pil', label='Upload an image'),
                     outputs='label')

# Launch the web app
iface.launch()

In this example, we define a function predict that takes an image as input, preprocesses it, makes a prediction using the trained model, and returns the predicted class. We then create a Gradio interface using the gr.Interface class, passing in the predict function, an image input component, and a label output component. Finally, we launch the web app using the launch method. When you run this code, Gradio will start a local web server and open up a browser window with the web app. You can then upload an image and see the predicted class. Note that this is just a simple example to get you started. Gradio offers many more features and options to customize your web app, such as input and output types, labels, and styles, as well as options to share your app, collect data, and integrate with other tools and services.





## Second Example

In [ ]:
!pip install gradio tensorflow

In [ ]:
import tensorflow as tf
import gradio as gr
import requests

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

In [ ]:
def classify_image(image):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image.numpy().reshape(1, 224, 224, 3))
    predictions = tf.keras.applications.mobilenet_v2.predict(image)
    top_k = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top_k=3)[0]
    confidences = {labels[i]: float(predictions[0][i]) for i in range(1000)}
    return confidences, top_k

In [ ]:
iface = gr.Interface(fn=classify_image,
                     inputs=gr.Image(shape=(224, 224)),
                     outputs=gr.Label(num_top_classes=3),
                     examples=["banana.jpg", "car.jpg"])

iface.launch()

In [ ]:
import gradio as gr
import requests
import numpy as np
import tensorflow as tf

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def classify_image(image):
    if image is None or image.shape[0] == 0 or image.shape[1] == 0:
        return {"error": "Please upload a valid image file."}, None

    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image.numpy().reshape(1, 224, 224, 3))
    predictions = tf.keras.applications.mobilenet_v2.predict(image)
    top_k = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top_k=3)[0]
    confidences = {labels[i]: float(predictions[0][i]) for i in range(1000)}
    return confidences, top_k

iface = gr.Interface(fn=classify_image,
                     inputs=gr.Image(shape=(224, 224), label="Upload an image"),
                     outputs=gr.Label(num_top_classes=3),
                     examples=["banana.jpg", "car.jpg"])

iface.launch()

## Third example

In [ ]:
!pip install tensorflow streamlit pandas matplotlib sklearn

Next, you can create a Python script (e.g. app.py) with the following code:



In [ ]:
# Import necessary libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read\_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')

# Preprocess the data
X = df.drop('survived', axis=1).values
y = df['survived'].values
X_train, X_test, y_train, y_test = train\_test\_split(X, y, test\_size=0.2, random\_state=42)
scaler = StandardScaler()
X_train = scaler.fit\_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input\_shape=(X\_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary\_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X\_train, y\_train, epochs=10, batch\_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X\_test, y\_test)
st.write('Test accuracy:', accuracy)

# Make predictions
predictions = model.predict(X\_test)
predictions = np.round(predictions)

# Display the confusion matrix
st.write('Confusion Matrix:')
st.write(pd.crosstab(pd.Series(y\_test, name='Actual'), pd.Series(predictions, name='Predicted')))

# Display the first 5 rows of the test data
st.write('First 5 rows of the test data:')
st.write(df.iloc[:5])

# Display the model summary
st.write('Model summary:')
model.summary(print\_fn=lambda x: st.write(x))

This script will load the Titanic dataset from a public URL, preprocess the data, define and train a simple neural network model, and then display the test accuracy, confusion matrix, first 5 rows of the test data, and the model summary.

To deploy the model using Streamlit, you can run the following command in your terminal:

In [ ]:
streamlit run app.py

This will start a local Streamlit server, and you can view the web app by opening a web browser and navigating to http://localhost:8501.

Note that this is just a simple example, and you can customize the script to load your own dataset, preprocess the data differently, define a more complex model, and display additional information in the web app.

## Fourth example

Trained multiclassification task model using TensorFlow and Streamlit for deployment, you can follow these steps:

Load the pre-trained model and its weights.
Preprocess the input data to match the input shape and normalization requirements of the model.
Add a new classification head to the model and compile it with a suitable loss function, optimizer, and metrics.
Fine-tune the model by unfreezing some of the top layers and training them along with the new classification head.
Create a Streamlit app to deploy the model and allow users to interact with it.
Here's a sample code to get you started:

In [ ]:
import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image

# Load the pre-trained model and its weights
model = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3), include_top=False, weights='imagenet')

# Preprocess the input data
@st.cache
def preprocess_image(image):
    img = Image.open(image)
    img = img.resize((160, 160))
    img_array = np.array(img) / 255.0
    img_batch = np.expand_dims(img_array, axis=0)
    return img_batch

# Add a new classification head to the model and compile it
num_classes = 2  # Change this to the number of classes in your dataset
image_input = tf.keras.Input(shape=(160, 160, 3))
x = model(image_input)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
new_model = tf.keras.Model(image_input, output)
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
for layer in model.layers:
    layer.trainable = False
new_model.trainable = True

# Load the training and validation datasets
train_ds = ...  # Load your training dataset here
val_ds = ...  # Load your validation dataset here

# Fine-tune the model
new_model.fit(train_ds, validation_data=val_ds, epochs=10)

# Create a Streamlit app
def main():
    st.title("Image Classification Model")
    image_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
    if image_file is not None:
        img_batch = preprocess_image(image_file)
        prediction = new_model.predict(img_batch)
        class_index = np.argmax(prediction)
        class_name = ('cat', 'dog')[class_index]
        st.write(f"The image is likely a {class_name}.")
        st.image(image_file, caption=f"Classified as a {class_name}", use_column_width=True)

if __name__ == "__main__":
    main()

# Sentiment analysis task using deep learning and the xai for interpretation and deploy it using streamlit and tensorflow

Here is an example of code to train a deep learning model for a sentiment analysis task using Keras and TensorFlow, and use SHAP for interpretation. The model is then deployed using Streamlit.

First, let's start by importing the necessary libraries:

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import shap

Next, let's load and preprocess the data. For this example, I will use the IMDB movie reviews dataset which is included in Keras.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

# Preprocess the data
max_review_length = 120
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_length)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_length)

# Convert labels to binary
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

Next, let's define and train the model. I will use a simple LSTM model for this example.



In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(10000, 128, input_length=max_review_length),
    keras.layers.LSTM(128),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3)

Next, let's use SHAP to explain the model's predictions. I will use the Keras explainer provided by SHAP.

In [ ]:
explainer = shap.DeepExplainer(model, x_train[:10])
shap_values = explainer.shap_values(x_train[:10])

Finally, let's deploy the model using Streamlit.



In [ ]:
import streamlit as st

def predict(model, text):
    # Preprocess the text
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    word_indices = [keras.datasets.imdb.get_word_index()[word] for word in words if word in keras.datasets.imdb.get_word_index().keys()]
    word_indices = [wi - 3 for wi in word_indices if wi > 2]
    word_indices = pad_sequences([word_indices], maxlen=max_review_length)
    # Make prediction
    prediction = model.predict(word_indices)
    return prediction

st.title("Sentiment Analysis")
st.write("Enter a movie review to classify its sentiment")

text = st.text_input("Review")
if st.button("Predict"):
    with st.spinner("Predicting..."):
        prediction = predict(model, text)
        if prediction > 0.5:
            st.write("Positive review")
        else:
            st.write("Negative review")

This is a basic example, you can add more feature like visualization, explanation, and more.

Please note that this is a simplified example and you may need to adjust the code to fit your specific use case. Also, you need to install the necessary libraries and dependencies before running the code

### Enhancement
To enhance and add more advanced features to a deep learning model for sentiment analysis, and deploy it using Streamlit, TensorFlow, and XAI for interpretation, you can follow the steps below:

Install necessary libraries:

In [ ]:
# Install necessary libraries:

!pip install streamlit tensorflow transformers xai-toolkit

In [ ]:
# Create a new Python file (e.g., app.py) and import the necessary libraries:

import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from xai_toolkit import Explanation

# Load the pre-trained model and tokenizer:

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
# Define a function to preprocess the input text and generate predictions:

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities)
    confidence = probabilities[0, predicted_class].item()
    return predicted_class.item(), confidence

# Define a function to generate explanations using XAI:

def generate_explanation(text):
    inputs = tokenizer(text, return_tensors="pt")
    explanation = Explanation(model, inputs)
    return explanation.get_integrated_gradients()


# Create the Streamlit app:
st.title("Sentiment Analysis with XAI")
text = st.text_input("Enter your text:")

if text:
    class_id, confidence = predict_sentiment(text)
    sentiment = "Positive" if class_id == 0 else "Negative"
    st.write(f"Sentiment: {sentiment} (Confidence: {confidence:.2f})")

    explanation = generate_explanation(text)
    st.write("Explanation:")
    st.image(explanation.get_heatmap(), caption="Heatmap", use_column_width=True)

In [ ]:
# Deploy the app using Streamlit Sharing:

streamlit run app.py --browser.server.headless

After deploying, you can share the link with the Product Manager or any other non-technical person. They can enter their text, and the model will classify it as 'POSITIVE' or 'NEGATIVE' sentiment, along with a confidence score. Additionally, the XAI-generated explanation will help them understand the model's decision

## Enhancement 2

In [ ]:
# Import necessary libraries
import streamlit as st
import tensorflow as tf
import numpy as
import pandas pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from pprint import pprint
import plotly.graph_objects as go
from wordcloud import WordCloud

# Load the dataset
@st.cache
def load_data():
    url = "https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv"
    data = pd.read_csv(url)
    return data

data = load_data()

# Preprocess the data
def preprocess_data(data):
    # Remove HTML tags
    data.review = data.review.str.replace(r'<.*?>', '')

    # Remove non-alphabetic characters
    data.review = data.review.str.replace(r'[^a-zA-Z\s]', '')

    # Convert to lowercase
    data.review = data.review.str.lower()

    # Remove stopwords
    stopwords = set(stopwords.words('english'))
    data.review = data.review.apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))

    # Tokenize the reviews
    max_fatures = 2000
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(data.review)
    sequences = tokenizer.texts_to_sequences(data.review)
    data.review = sequences

    # Pad sequences
    X = pad_sequences(data.review, maxlen=max_fatures)
    return X, tokenizer

X, tokenizer = preprocess_data(data)

# Preprocess the labels
le = LabelEncoder()
data.sentiment = le.fit_transform(data.sentiment)
y = to_categorical(data.sentiment)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
def define_model():
    model = tf.keras.Sequential()
    model.add(Embedding(max_fatures, 128, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(192, dropout=0.4, recurrent_dropout=0.4))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Load the saved model
model = load_model('model.h5')

# Define a function for predicting the sentiment of a review
def predict_sentiment(review):
    review = [tokenizer.word_index[word] for word in review.split() if word in tokenizer.word_index]
    review = pad_sequences([review], maxlen=X.shape[1])
    prediction = model.predict(review)
    if prediction > 0.5:
        return 'Positive'
    else:
        return 'Negative'

# Define a function for interpreting the model's predictions using SHAP
def explain_prediction(review):
    explainer = shap.DeepExplainer(model, X_train)
    shap_values = explainer.shap_values(review)
    return shap_values

# Define a function for visualizing the word cloud of a review
def visualize_word_cloud(review):
    stopwords = set(stopwords.words('english'))
    review = " ".join(review for review in review.split() if review not in stopwords)
    wordcloud = WordCloud(width=800, height=400, random_state=21, max_font_size=110).generate(review)
    fig = go.Figure(data=go.Scatter(x=0, y=0, mode='lines', marker=dict(size=10, color='850000', opacity=0.6)))
    fig.add_shape(type='circle', xref='x', yref='y', x0=0, x1=1, y0=0, y1=1, line=dict(color='850000'))
    fig.add_trace(go.Scatter(x=wordcloud.words[:, 0], y=wordcloud.words[:, 1], mode='text', text=wordcloud.words[:, 0], textfont=dict(size=wordcloud.words[:, 2])))
    st.plotly_chart(fig)

# Define the Streamlit app
def main():
    st.title('Sentiment Analysis using Deep Learning and XAI')

    # Text input for user review
    user_review = st.text_area('Enter your review here:')

    if st.button('Predict Sentiment'):
        with st.spinner('Predicting sentiment...'):
            sentiment = predict_sentiment(user_review)
            st.success(f'The sentiment of the review is {sentiment}.')

            # Interpret the model's predictions
            if st.checkbox('Interpret Prediction'):
                with st.spinner('Interpreting prediction...'):
                    shap_values = explain_prediction(pad_sequences([user_review], maxlen=X.shape[1]))
                    st.write('SHAP values:')
                    st.write(shap_values)

                    # Visualize the word cloud
                    visualize_word_cloud(user_review)

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import shap
from wordcloud import WordCloud
import plotly.graph_objects as go
import streamlit as st

# Define a function for loading the data
def load_data(file):
    df = pd.read_csv(file)
    df['Text'] = df['Text'].apply(lambda x: x.lower())
    df['Text'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in ('.', ',', '!', '?', ';', ':')])))
    return df

# Define a function for building the model
def build_model(max_features, embedding_dims, maxlen):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_features, embedding_dims, input_length=maxlen),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define a function for predicting the sentiment
def predict_sentiment(text):
    predictions = model.predict(np.array([text]))
    return 'Positive' if predictions[0][0] > 0.5 else 'Negative'

# Define a function for interpreting the model's predictions
def explain_prediction(text):
    # Explain predictions using SHAP
    explainer = shap.Explainer(model, X)
    shap_values = explainer.shap_values(text)
    return shap_values

# Define a function for visualizing the word cloud of a review
def visualize_word_cloud(review):
    stopwords = set(stopwords.words('english'))
    review = " ".join(review for review in review.split() if review not in stopwords)
    wordcloud = WordCloud(width=800, height=400, random_state=21, max_font_size=110).generate(review)
    fig = go.Figure(data=go.Scatter(x=0, y=0, mode='lines', marker=dict(size=10, color='850000', opacity=0.6)))
    fig.add_shape(type='circle', xref='x', yref='y', x0=0, x1=1, y0=0, y1=1, line=dict(color='850000'))
    fig.add_trace(go.Scatter(x=wordcloud.words[:, 0], y=wordcloud.words[:, 1], mode='text', text=wordcloud.words[:, 0], textfont=dict(size=wordcloud.words[:, 2])))
    st.plotly_chart(fig)

# Define the Streamlit app
def main():
    st.title('Sentiment Analysis using Deep Learning and XAI')

    # Load the data
    data = load_data('data.csv')

    # Create the training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Sentiment'], test_size=0.2, random_state=42)

    # Tokenize the data and pad the sequences
    tokenizer = Tokenizer(num_words=5000, oov_token='OOV')
    tokenizer.fit_on_texts(X_train)
    X_train = tokenizer.texts_to_sequences(X_

# Sentiment analysis task using deep learning and the xai for interpretation and deploy it using Gradio and tensorflow

Here is an example of how you can train a deep learning model for a sentiment analysis task using TensorFlow and the Keras API, interpret the model using XAI techniques, and deploy it using Gradio:

First, let's start by importing the necessary libraries:

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import gradio as gr
import shap
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Next, let's load the dataset. For this example, I will use the IMDB movie reviews dataset, which is a binary sentiment analysis dataset.

data = pd.read_csv('imdb_reviews.csv')
data = data.sample(frac=1).reset_index(drop=True)

# Split the data into training and testing sets
train_data = data[:20000]
test_data = data[20000:]

# Define the labels
train_labels = train_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values
test_labels = test_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values

# Remove the sentiment column
train_data = train_data.drop('sentiment', axis=1)
test_data = test_data.drop('sentiment', axis=1)

Next, let's preprocess the text data by tokenizing the words and converting them into sequences. We will also pad the sequences so that they all have the same length.

In [ ]:
# Tokenize the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['review'])

# Convert the words to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['review'])
test_sequences = tokenizer.texts_to_sequences(test_data['review'])

# Pad the sequences
max_sequence_length = 1000

x_train = pad_sequences(train_sequences, maxlen=max_sequence_length)
x_test = pad_sequences(test_sequences, maxlen=max_sequence_length)


In [ ]:
# Next, let's define the model architecture. For this example, I will use a simple LSTM model.

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_sequence_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, train_labels, validation_data=(x_test, test_labels), epochs=10)

Next, let's interpret the model using SHAP (SHapley Additive exPlanations). SHAP is a method for interpreting the output of machine learning models by quantifying the importance of each feature in making a specific prediction.

In [ ]:
# Generate SHAP values for the training set
explainer = shap.DeepExplainer(model, x_train)
shap_values = explainer.shap_values(x_train)

# Visualize the SHAP values for the first 10 samples
shap.force_plot(explainer.expected_value[0], shap_values[0][:10], train_data['review'][:10])

Finally, let's deploy the model using Gradio. Gradio is a Python library for creating machine learning demos that can be shared with others.

In [ ]:
# Define the function that takes in the input and returns the prediction
def predict(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    prediction = model.predict(padded_sequence)
    return 'Positive' if prediction > 0.5 else 'Negative'

# Create the Gradio interface
iface = gr.Interface(fn=predict, inputs='text', outputs='label')

# Launch the interface
iface.launch()

This will create a web interface where users can input text and get a sentiment analysis prediction. The interface will look something like this:

Gradio Interface

And that's it! You have now trained a deep learning model for a sentiment analysis task, interpreted the model using XAI techniques, and deployed it using Gradio. Of course, this is just a simple example, and there are many ways to customize and improve the model and the interface. But hopefully, this gives you a good starting point for building your own sentiment analysis model using TensorFlow and Gradio

In [ ]:
# Additional code
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.client import predict_client

# Create a predict client
hostport = 'localhost:9000'
predict_client = predict_client.PredictClient(hostport)

# Define the model name and signature name
model_name = 'my_sentiment_analysis_model'
signature_name = 'serving_default'

# Load the model into memory
model = tf.keras.models.load_model('my_sentiment_analysis_model.h5')

# Define the input data
input_data = {'text': ['I love this product!']}

# Convert the input data to a serialized TensorFlow Serving message
input_data_serialized = tf.compat.as_bytes(tf.io.serialize_tensor(tf.convert_to_tensor(input_data)))

# Define the request message
request = predict_pb2.PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = signature_name
request.inputs['input'].CopyFrom(tf.make_tensor_proto(input_data_serialized))

# Send the request to the TensorFlow Serving server
response = predict_client.Predict(request, timeout=10.0)

# Extract the output data from the response message
output_data = tf.make_ndarray(response.outputs['output'])

# Print the output data
print(output_data)

## Enhancement

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary resources
nltk.download('stopwords')
nltk.download('wordnet')

# Define functions for data preprocessing
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(words)
    return text

# Preprocess the text data
train_data['text'] = train_data['text'].apply(preprocess_text)
test_data['text'] = test_data['text'].apply(preprocess_text)

In [ ]:
# Feature Engineering:


from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TfidfVectorizer with custom parameters
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 3))

# Generate features for the text data
train_features = vectorizer.fit_transform(train_data['text'])
test_features = vectorizer.transform(test_data['text'])

In [ ]:
# Model Architecture:

from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# Load a pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(train_features) * epochs)

# Define training function
def train(model, optimizer, scheduler, train_features, train_labels, epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_features, labels=train_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    scheduler.step()
    return loss.item()

# Define evaluation function
def evaluate(model, test_features, test_labels):
    model.eval()
    with torch.no_grad():
        outputs = model(test_features)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        acc = accuracy_score(test_labels, preds)
        f1 = f1_score(test_labels, preds, average='macro')
    return acc, f1

# Train the model
for epoch in range(epochs):
    train_loss = train(model, optimizer, scheduler, train_features, train_labels, epochs)
    test_acc, test_f1 = evaluate(model, test_features, test_labels)
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.3f}, Test Acc: {test_acc:.3f}, Test F1: {test_f1:.3f}')

In [ ]:
# Explainability (XAI):

import shap

# Generate SHAP values for the test data
explainer = shap.DeepExplainer(model, test_features)
shap_values = explainer.shap_values(test_features)

# Visualize the SHAP values for a few instances
for i in range(10):
    shap.force_plot(explainer.expected_value[i], shap_values[i], test_features.numpy()[i])
    plt.show()


### Deployment

In [ ]:
!pip install gradio transformers torch scipy

In [ ]:
import gradio as gr
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from scipy.special import softmax

In [ ]:
model_path = "KAITANY/finetuned-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
def preprocess(text):
    # Preprocess text (username and link placeholders)
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
def sentiment_analysis(text):
    text = preprocess(text)
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    # Make a prediction
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the predicted class probabilities
    scores = torch.softmax(outputs.logits, dim=1).tolist()[0]
    # Map the scores to labels
    labels = ['Negative', 'Neutral', 'Positive']
    scores_dict = {label: score for label, score in zip(labels, scores)}
    return scores_dict

In [ ]:
title = "Sentiment Analysis Application\n\n\nThis application assesses if a twitter post relating to vaccination is positive,neutral or negative"
demo = gr.Interface(fn=sentiment_analysis,
                   inputs=gr.Textbox(placeholder="Write your tweet here..."),
                   outputs=gr.Label(num_top_classes=3),
                   examples=[["The Vaccine is harmful!"],
                              ["I cant believe people don't vaccinate their kids"],
                              ["FDA think just not worth the AE unfortunately"],
                              ["For a vaccine given to healthy"]],
                   title=title)
demo.launch(share=True)

**Interpretability with XAI**

To interpret the predictions made by the model, we can use explainable AI (XAI) techniques. One such technique is Local Interpretable Model-agnostic Explanations (LIME), which can help us understand how the model is making its predictions.

Let's install the necessary library:

In [ ]:
!pip install alibi

In [ ]:
from alibi.explainers import LIME, TaskType

In [ ]:
# Let's define a function to explain the predictions:

def explain_prediction(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    # Make a prediction
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the predicted class probabilities
    scores = torch.softmax(outputs.logits, dim=1).tolist()[0]
    # Map the scores to labels
    labels = ['Negative', 'Neutral', 'Positive']
    # Explain the prediction using LIME
    explainer = LIME(model, task="classification", feature_type="text", categorical_features=[])
    explanation = explainer.explain(text, labels)
    return explanation

In [ ]:
# Now, let's modify the Gradio interface to include the explanation:

title = "Sentiment Analysis Application\n\n\nThis application assesses if a twitter post relating to vaccination is positive,neutral or negative"
demo = gr.Interface(fn=sentiment_analysis,
                   inputs=gr.Textbox(placeholder="Write your tweet here..."),
                   outputs=gr.Label(num_top_classes=3),
                   examples=[["The Vaccine is harmful!"],
                              ["I cant believe people don't vaccinate their kids"],
                              ["FDA think just not worth the AE unfortunately"],
                              ["For a vaccine given to healthy"]],
                   output_types=[gr.outputs.Label(num_top_classes=3)],
                   output_styles=[{"type": "prediction", "prediction_type": "probability"}],
                   description="This application assesses if a twitter post relating to vaccination is positive,neutral or negative. You can also see the explanation for the prediction.",
                   article="The explanation for the prediction is generated using Local Interpretable Model-agnostic Explanations (LIME). It shows the importance of each word in the input text for the predicted sentiment.",
                   sidebar="Explain Prediction",
                   sidebar_outputs=[gr.outputs.Textbox(label="Explanation")],
                   sidebar_function=explain_prediction,
                   title=title)
demo.launch(share=True)